# Basic Bayesian Linear Regression Implementation

In [41]:
!pip list | grep pymc
!pip list | grep arviz
!pip list | grep bambi

pymc                              5.25.1
arviz                             0.22.0
bambi                             0.15.0


In [42]:
#!pip install pymc #==5.0.0
#!pip install arviz
!pip install bambi

In [43]:
import pymc as pm
import arviz as az
import bambi as bmb

# Pandas and numpy for data manipulation
import scipy
from sklearn.linear_model import LinearRegression
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

# Matplotlib and seaborn for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# DOCUMENTACIÓN DE LA BIBLIOTECA PYMC
# https://docs.pymc.io/en/v3/pymc-examples/examples/getting_started.html
# https://docs.pymc.io/en/v3/pymc-examples/examples/generalized_linear_models/GLM-linear.html
# https://www.pymc.io/welcome.html

# SOBRE NUTS: https://arxiv.org/abs/1111.4246

# EJERCICIO BASADO EN EL SGTE ARTÍCULO:
# https://towardsdatascience.com/introduction-to-bayesian-linear-regression-e66e60791ea7

# OTRO LINK DE INTERÉS:
# https://towardsdatascience.com/bayesian-linear-regression-in-python-using-machine-learning-to-predict-student-grades-part-2-b72059a8ac7e

# Linear Regression to verify implementation
# Scipy for statistics
# PyMC for Bayesian Inference

warnings.filterwarnings('ignore')
print(f"Running on PyMC v{pm.__version__}")


Running on PyMC v5.25.1


# Load in Exercise Data

In [44]:
url = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/arbolado-en-espacios-verdes/arbolado-en-espacios-verdes.csv'
arbolado = pd.read_csv(url)

# columnas que me interesan
col_interes = ['diametro', 'altura_tot', 'nombre_com']
nombres_interes = ['Jacarandá', 'Palo borracho rosado', 'Eucalipto', 'Ceibo']

# preparo los dataframe
# selecciono x columnas
data1 = arbolado[col_interes].copy()
# cambio nombre a 'altura'
data1.rename(columns={'altura_tot': 'altura'}, inplace=True)
data2 = data1[data1['nombre_com'].isin(nombres_interes)]


In [45]:
# me quedo con el árbol que me interese
mi_arbol='Eucalipto'
arbol = data2[data2['nombre_com'] == mi_arbol]
arbol.reset_index(inplace=True)
print(arbol.head())
arbol.drop(['nombre_com', 'index'], axis=1, inplace=True)
arbol.tail()

   index  diametro  altura nombre_com
0    363        40      20  Eucalipto
1    364        40      20  Eucalipto
2    365        40      20  Eucalipto
3    366        40      20  Eucalipto
4    367        40      20  Eucalipto


,diametro,altura
4107,74,23
4108,40,7
4109,70,20
4110,40,10
4111,100,19


In [46]:
arbol.shape

(4112, 2)

In [47]:
print(arbolado.shape)
print(arbolado.columns)
print(arbolado['nombre_com'].unique())
print(arbolado['nombre_com'].unique().size)

(51502, 17)
Index(['long', 'lat', 'id_arbol', 'altura_tot', 'diametro', 'inclinacio',
       'id_especie', 'nombre_com', 'nombre_cie', 'tipo_folla', 'espacio_ve',
       'ubicacion', 'nombre_fam', 'nombre_gen', 'origen', 'coord_x',
       'coord_y'],
      dtype='object')
['Washingtonia (Palmera washingtonia)' 'Ombú' 'Catalpa' 'Ceibo'
 'Brachichiton (Árbol botella, Brachichito)' 'Álamo plateado'
 'Acacia de constantinopla' 'Acacia' 'Roble sedoso (Grevillea)' 'Arce'
 'Palo borracho rosado' 'Álamo negro del Canada' 'Jacarandá'
 'Álamo carolina' 'Aguaribay' 'Paraíso' 'Espinillo (Aromo)' 'Eucalipto'
 'Ciprés leylandi' 'Ciprés' 'No Determinable' 'Álamo' 'No Determinado'
 'Cedro' 'Lapacho' 'Crataegus' 'Cedro de San Juan' 'Juniperus'
 'Chamaecyparis' 'Álamo blanco piramidal' 'Podocarpus' 'Tuja' 'El chañar'
 'Arce tridente' 'Citricos' 'Sauce' 'Viburnum dulce' 'Corona de cristo'
 'Taxodium' 'Mirto' 'Árbol del humo' 'Celtis tala'
 'Criptomeria (Cedro del Japón)' 'Coculus, Cóculo' 'Caoba del sur'

In [48]:
arbolado

,long,lat,id_arbol,altura_tot,diametro,inclinacio,id_especie,nombre_com,nombre_cie,tipo_folla,espacio_ve,ubicacion,nombre_fam,nombre_gen,origen,coord_x,coord_y
0,-58.477564,-34.645015,1,6,35,0,53,Washingtonia (Palmera washingtonia),Washingtonia filifera,Palmera,"AVELLANEDA, NICOLÁS, Pres.","DIRECTORIO, AV. - LACARRA, AV. - MONTE - AUTO...",Arecaceas,Washingtonia,Exótico,98692.305719,98253.300738
1,-58.477559,-34.645047,2,6,35,0,53,Washingtonia (Palmera washingtonia),Washingtonia filifera,Palmera,"AVELLANEDA, NICOLÁS, Pres.","DIRECTORIO, AV. - LACARRA, AV. - MONTE - AUTO...",Arecaceas,Washingtonia,Exótico,98692.751564,98249.733979
2,-58.477551,-34.645091,3,6,35,0,53,Washingtonia (Palmera washingtonia),Washingtonia filifera,Palmera,"AVELLANEDA, NICOLÁS, Pres.","DIRECTORIO, AV. - LACARRA, AV. - MONTE - AUTO...",Arecaceas,Washingtonia,Exótico,98693.494639,98244.829684
3,-58.478129,-34.644567,4,17,50,0,65,Ombú,Phytolacca dioica,Árbol Latifoliado Caducifolio,"AVELLANEDA, NICOLÁS, Pres.","DIRECTORIO, AV. - LACARRA, AV. - MONTE - AUTO...",Fitolacáceas,Phytolacca,Nativo/Autóctono,98640.439091,98302.938142
4,-58.478121,-34.644598,5,17,50,0,65,Ombú,Phytolacca dioica,Árbol Latifoliado Caducifolio,"AVELLANEDA, NICOLÁS, Pres.","DIRECTORIO, AV. - LACARRA, AV. - MONTE - AUTO...",Fitolacáceas,Phytolacca,Nativo/Autóctono,98641.182166,98299.519997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51497,-58.505752,-34.562998,51725,15,30,0,45,Casuarina,Casuarina cunninghamiana,Árbol Latifoliado Perenne,GENERAL PAZ,"LARRALDE, CRISOLOGO, AV. - PAZ, GRAL., AV.- AI...",Casuarinaceas,Casuarina,Exótico,96104.125550,107350.962385
51498,-58.507111,-34.565843,51726,13,31,0,45,Casuarina,Casuarina cunninghamiana,Árbol Latifoliado Perenne,GENERAL PAZ,"LARRALDE, CRISOLOGO, AV. - PAZ, GRAL., AV.- AI...",Casuarinaceas,Casuarina,Exótico,95979.513368,107035.237268
51499,-58.475721,-34.565192,51727,11,28,0,17,Árbol del cielo (Ailanto o Árbol de los dioses),Ailanthus altissima,Árbol Latifoliado Caducifolio,"CAMPELO, LICENCIADO CARLOS",ESTACIÓN COGHLAN - Propiedad particular e/UGAR...,Simarrubáceas,Ailanthus,Exótico,98860.090864,107108.347622
51500,-58.507026,-34.565670,51728,5,32,0,342,Ciprés,Cupressus sp.,Árbol Conífero Perenne,GENERAL PAZ,"LARRALDE, CRISOLOGO, AV. - PAZ, GRAL., AV.- AI...",Cupresáceas,Cupressus,Exótico,95987.288841,107054.471332


In [49]:
arbol

,diametro,altura
0,40,20
1,40,20
2,40,20
3,40,20
4,40,20
...,...,...
4107,74,23
4108,40,7
4109,70,20
4110,40,10


In [50]:
plt.figure(figsize=(8, 8))
plt.plot(arbol['diametro'], arbol['altura'], 'bo', alpha=0.3)
plt.xlabel('Diámetro (cm)', size=18)
plt.ylabel('Altura (m)', size=18)
plt.title(f'{mi_arbol}: Altura vs. Diámetro', size=20)


Text(0.5, 1.0, 'Eucalipto: Altura vs. Diámetro')

In [51]:
arbol.describe().T

,count,mean,std,min,25%,50%,75%,max
diametro,4112.0,54.362354,30.043462,2.0,32.0,43.0,68.0,305.0
altura,4112.0,21.647617,6.489263,1.0,20.0,21.0,25.0,52.0


In [ ]:
# En este bloque se crean las variables para el modelo de regresión:
X = arbol.loc[:, ['diametro']]
y = arbol.loc[:, 'altura']


# Implement Ordinary Least Squares Linear Regression by Hand

In [53]:
def ajuste_lineal_simple(x, y):
    ''' Dados dos np.array 'x' e 'y', calcula por el método de mínimos cuadrados
    el ajuste lineal para los np.array dados. Los coeficientes son retornados
    en una tupla (pendiente,ordenada al origen).'''
    a = sum(((x - x.mean())*(y-y.mean()))) / sum(((x-x.mean())**2))
    b = y.mean() - a*x.mean()

    return a, b


In [54]:
# Run the by hand implementation
xs = arbol[['diametro']].to_numpy()
ys = arbol[['altura']].to_numpy()
by_hand_coefs = ajuste_lineal_simple(xs, ys)
print('Intercept calculated by hand:', float(by_hand_coefs[1]))
print('Slope calculated by hand: ', float(by_hand_coefs[0]))


Intercept calculated by hand: 16.567984270345292
Slope calculated by hand:  0.09344025928629655


In [60]:
xss = np.linspace(0, 200, 1000)
yss = by_hand_coefs[1] + by_hand_coefs[0] * xss

plt.figure(figsize=(8, 8))
plt.plot(arbol['diametro'], arbol['altura'],
         'bo', label='observations', alpha=0.3)
plt.xlabel('Diámetro (cm)', size=18)
plt.ylabel('Altura (m)', size=18)
plt.plot(xss, yss, 'r--', label='OLS Fit', linewidth=3)
plt.legend(prop={'size': 16})
plt.title(f'Altura vs Diámetro {mi_arbol}', size=20)


Text(0.5, 1.0, 'Altura vs Diámetro Eucalipto')

## Prediction for Datapoint

In [56]:
print(
    f'Altura estimada para un diámetro de 100 cm: {round(float(by_hand_coefs[1] + by_hand_coefs[0] * 100), 2)} m')


Altura estimada para un diámetro de 100 cm: 25.91 m


# Verify with Scikit-learn Implementation

In [57]:
# Create the model and fit on the data
lr = LinearRegression()
lr.fit(X['diametro'].to_frame(), y)
print('Intercept from library:', lr.intercept_)
print('Slope from library:', lr.coef_[0])


Intercept from library: 16.567984270345242
Slope from library: 0.09344025928629748


# Bayesian Linear Regression

### PyMC for Bayesian Inference

Implement MCMC to find the posterior distribution of the model parameters. Rather than a single point estimate of the model weights, Bayesian linear regression will give us a posterior distribution for the model weights.

In [58]:
# https://www.pymc.io/welcome.html
# https://www.pymc.io/projects/examples/en/latest/gallery.html

# primero estandarizamos los datos

def standardize(x):
    return (x - x.mean()) / x.std()


X = standardize(X['diametro'])
y = standardize(y)

In [59]:
plt.figure(figsize=(8, 8))
plt.plot(X, y, 'bo', alpha=0.3)
plt.xlabel('Diámetro estandarizado', size=18)
plt.ylabel('Altura estandarizada', size=18)
plt.title(f'{mi_arbol} Estandarizado: Altura vs. Diámetro', size=20)

Text(0.5, 1.0, 'Eucalipto Estandarizado: Altura vs. Diámetro')

In [39]:
# https://www.pymc.io/welcome.html
# https://www.pymc.io/projects/examples/en/latest/gallery.html

semilla = 16  # Fija la semilla para reproducibilidad

# MCMC es un algoritmo Markov Chain Montecarlo
with pm.Model() as lm:  # Define el contexto del modelo bayesiano en PyMC

    # PRIMERO DEFINE LOS PRIORS, ES DECIR, LO SUPUESTO A PRIORI
    # Intercept: prior normal con media 0 y desviación estándar 1
    intercept = pm.Normal('Intercept', mu=0, sigma=1)

    # Slope: prior normal con media 0 y desviación estándar 1
    slope = pm.Normal('slope', mu=0, sigma=1)

    # Standard deviation: prior half-normal (solo valores positivos)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Estimate of mean: define la media del modelo lineal
    mean = intercept + slope * X.values

    # Observed values, DEBE SER DATAFRAME
    # Define la variable observada como normal, centrada en la media calculada y con desviación sigma
    Y_obs = pm.Normal('Y_obs', mu=mean, sigma=sigma, observed=y.values)

    # Genera muestras predictivas a priori para los parámetros del modelo
    datos = pm.sample_prior_predictive(draws=4000,

Sampling: [Intercept, Y_obs, sigma, slope]


In [24]:
datos

Inference data with groups:
	> prior
	> prior_predictive
	> observed_data

ALTERNATIVA PARA INSTANCIAR MODELOS
Queda una sintaxis super compacta :)

In [25]:
# Usamos Bambi

model = bmb.Model('altura ~ diametro', arbol)
predictor = model.fit(draws=4000)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, diametro]


Output()

Sampling 4 chains for 1_000 tune and 4_000 draw iterations (4_000 + 16_000 draws total) took 31 seconds.


In [26]:
predictor

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

# Bayesian Model Results

The Bayesian Model provides more opportunities for interpretation than the ordinary least squares regression because it provides a posterior distribution. We can use this distribution to find the most likely single value as well as the entire range of likely values for our model parameters.

PyMC has many built in tools for visualizing and inspecting model runs. These let us see the distributions and provide estimates with a level of uncertainty, which should be a necessary part of any model.

In [27]:
with lm:
    datos.extend(pm.sample(1000, tune=2000, random_seed=semilla))

az.plot_trace(datos, figsize=(12, 12), combined=True)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, slope, sigma]


Output()

Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 25 seconds.


array([[<Axes: title={'center': 'Intercept'}>,
        <Axes: title={'center': 'Intercept'}>],
       [<Axes: title={'center': 'slope'}>,
        <Axes: title={'center': 'slope'}>],
       [<Axes: title={'center': 'sigma'}>,
        <Axes: title={'center': 'sigma'}>]], dtype=object)

## Posterior Distribution of Model Parameters

In [28]:
with lm:
    pm.sample_posterior_predictive(datos, extend_inferencedata=True, random_seed=semilla)


Sampling: [Y_obs]


Output()

In [29]:
datos.posterior_predictive

<xarray.Dataset> Size: 132MB
Dimensions:      (chain: 4, draw: 1000, Y_obs_dim_0: 4112)
Coordinates:
  * chain        (chain) int64 32B 0 1 2 3
  * draw         (draw) int64 8kB 0 1 2 3 4 5 6 ... 993 994 995 996 997 998 999
  * Y_obs_dim_0  (Y_obs_dim_0) int64 33kB 0 1 2 3 4 ... 4107 4108 4109 4110 4111
Data variables:
    Y_obs        (chain, draw, Y_obs_dim_0) float64 132MB 0.03547 ... 1.153
Attributes:
    created_at:                 2025-08-28T02:42:21.549230+00:00
    arviz_version:              0.22.0
    inference_library:          pymc
    inference_library_version:  5.25.1

In [30]:
az.plot_ppc(datos, num_pp_samples=100)

<Axes: xlabel='Y_obs'>

## Confidence Intervals for Model Parameters

In [31]:
with lm:
    pm.plot_forest(datos)


In [32]:
post=datos.posterior
X_sample=np.random.choice(X, 1000)
mu_pp=post['Intercept']+post['slope']*X_sample

In [33]:
mu_pp.shape

(4, 1000)

In [34]:
mu_pp

<xarray.DataArray (chain: 4, draw: 1000)> Size: 32kB
array([[ 0.48358363, -0.27475977, -0.05566202, ..., -0.18593026,
        -0.03845415, -0.32673521],
       [ 0.55633461, -0.28958276, -0.06315326, ..., -0.18864523,
        -0.04908304, -0.30615284],
       [ 0.52505209, -0.31651797, -0.05436741, ..., -0.18018053,
        -0.06266926, -0.35521546],
       [ 0.5027031 , -0.29672354, -0.05977344, ..., -0.16334755,
        -0.06783065, -0.34133544]])
Coordinates:
  * chain    (chain) int64 32B 0 1 2 3
  * draw     (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999

In [35]:
mu_pp.mean(('chain', 'draw'))

<xarray.DataArray ()> Size: 8B
array(0.0142052)

In [36]:
datos

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> prior_predictive
	> observed_data

In [37]:
plt.figure(figsize=(8, 8))
_, ax = plt.subplots()

#ax.plot(X, mu_pp.mean(("chain", "draw")), label="Curva promedio", color="C1", alpha=0.6)
ax.scatter(X, datos.observed_data["Y_obs"])
az.plot_hdi(X, datos.posterior_predictive["Y_obs"])
plt.title('Posterior Predictions with all Observations', size=20)
ax.set_xlabel('Diámetro Std (cm)', size=18)
ax.set_ylabel('Altura Std(m)', size=18)


Text(0, 0.5, 'Altura Std(m)')

In [38]:
with lm:
    summ = pm.summary(datos)
summ

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,0.000,0.014,-0.026,0.027,0.0,0.0,6033.0,3011.0,1.0
slope,0.432,0.014,0.407,0.461,0.0,0.0,5804.0,3060.0,1.0
sigma,0.902,0.010,0.884,0.922,0.0,0.0,6122.0,3158.0,1.0
